In [ ]:
pip install requests

In [ ]:
pip install progress_bar

In [ ]:
pip install selenium

In [ ]:
pip install beautifulsoup4

In [2]:
# import necessary modules
import time
import datetime
import random
import numpy as np
import base64

import re
import os

import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

from tqdm import tqdm
import urllib.request 
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.expected_conditions import presence_of_element_located
# browser.quit() # close the whole browser
# browser.close() # close the tab

In [1]:
def get_product_info_from_mainpage(URL):
    product_links = []
    image_links = []
    prices = []
    
    driver = webdriver.Chrome('C:/Users/Brian/Downloads/chromedriver_win32/chromedriver.exe')
    driver.implicitly_wait(200)
    
    try:
        driver.get(URL)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        time.sleep(60)
        
        product_cards = soup.find_all('div', {"class" : "ProductCard-container ProductCard-container--contained"})
        while len(product_cards) <= 0:
            print("CAPTCHA ENCOUNTERED. FIX")
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            product_cards = soup.find_all('div', {"class" : "ProductCard-container ProductCard-container--contained"})
            time.sleep(30)
        
        print("Number of products scraped from the page: ", len(product_cards))
        i = 0
        for product_card in product_cards:
            try:
                #get image link
                driver.implicitly_wait(10)
                image = product_card.find('div', {"class" : "BrowseProductCard-imageWrap"})
                #image_link = image.find('img')["src"]
                image_link = image.find('img')["srcset"].split(',')[-1].split(" ")[0]
                image_links.append(image_link)
                
                
                #get product link
                product_link = product_card.find('a')['href']
                product_links.append(product_link)                
                
                
                #get price
                price_block = product_card.find('div', {"class" : "ProductCard-pricing"})
                price = price_block.find('span').text
                
                '''
                try:
                    price = price_block.find('span', {"class" : "ProductCard-price"}).text
                except:
                    price = price_block.find('span', {"class" : "ProductCard-price ProductCard-price--sale"}).text
                '''
                
                prices.append(price)
                i += 1
                
                #Print summary information
                print(f"Item: {i}\n")
                print(f"Product link: {product_link}\n")
                print(f"Image link: {image_link}\n")
                print(f"Price: {price}\n")
                print("==============================================")
            
            except:
                print("Error")
                pass
        driver.close()
        print("Lengths of lists: ", len(product_links), len(image_links), len(prices))
        return (product_links, image_links, prices)
    
    except TimeoutException as e:
        print("Page Load Timeout Occurred. Quitting...")
        driver.close()

'''
def get_info_from_productlink(product_link):  #get the first image
    info = dict()
    page = requests.get(product_link)
    driver = webdriver.Chrome('C:/Users/Brian/Downloads/chromedriver_win32/chromedriver.exe')
    driver.get(product_link)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    time.sleep(random.randint(3,3))

    #price_block = soup.find('div', {"class" : "BasePriceBlock BasePriceBlock--highlight"})
    #price_block = soup.find('div', {"class" : "BasePriceBlock"})
    price_block = soup.find('span', {"class" : "notranslate"})
    price = price_block.text
    info['price'] = price
    #print(price)   
    
    image = soup.find('div', {'class' : "ImageComponent ImageComponent--overlay"})
    image_link = image.find('img')['src']
    info['img_url'] = image_link
    
    #print(f"Image link: {image_link}\n")
    #print(f"Link to product: {product_link}\n")

    driver.close()
    #browser.switch_to.window(browser.window_handles[0])
    return info
'''

'\ndef get_info_from_productlink(product_link):  #get the first image\n    info = dict()\n    page = requests.get(product_link)\n    driver = webdriver.Chrome(\'C:/Users/Brian/Downloads/chromedriver_win32/chromedriver.exe\')\n    driver.get(product_link)\n    soup = BeautifulSoup(driver.page_source, \'html.parser\')\n    time.sleep(random.randint(3,3))\n\n    #price_block = soup.find(\'div\', {"class" : "BasePriceBlock BasePriceBlock--highlight"})\n    #price_block = soup.find(\'div\', {"class" : "BasePriceBlock"})\n    price_block = soup.find(\'span\', {"class" : "notranslate"})\n    price = price_block.text\n    info[\'price\'] = price\n    #print(price)   \n    \n    image = soup.find(\'div\', {\'class\' : "ImageComponent ImageComponent--overlay"})\n    image_link = image.find(\'img\')[\'src\']\n    info[\'img_url\'] = image_link\n    \n    #print(f"Image link: {image_link}\n")\n    #print(f"Link to product: {product_link}\n")\n\n    driver.close()\n    #browser.switch_to.window(bro

In [ ]:
#TEST
get_product_info_from_mainpage('https://www.wayfair.com/filters/furniture/sb1/sofas-c413892-p2479~1000~*.html?itemsperpage=96&curpage=2')

In [3]:
def save_data(product_info_list, i):
    result = [[product[0], product[1]['price'], product[1]['img_url']] for product in product_info_list]
    df = pd.DataFrame(result)
    now = str(datetime.datetime.now()).replace(" ", "").replace(":", "").replace("-", "").split(".")[0][4:][:-2]
    df.to_csv(f'sofa_info_{str(i)}.csv', index=False, header=['item_no', 'price', 'img_url'])
    
def save_image_from_url(product, category):    
    bar_wrap = [None]

    item_no = product.name + 5780
    print(f"Product Row: {product.name}\n")
    base_name = "C:/Users/Brian/Desktop/ThePriceIsRight/" + category + "/"
    filename = f"./{category}/{item_no}.jpeg"
    filepath = base_name + str(item_no) + ".jpeg"
    url = str(product['img_url'])
    
    try:
        print(url)
        urllib.request.urlretrieve(url, filepath)
        return filename
    except:
        print(f"Failed to download image:{url}\n")
        return f"Failed to download image: {url}\n"
    
def save_images_from_csv(df):
    tqdm.pandas()
    df['filename'] = df.progress_apply(lambda x: save_image_from_url(x, category = 'sofa'), axis=1)
    return df

In [ ]:
product_info = []
#base_url = 'https://www.wayfair.com/furniture/sb0/sofas-c413892.html?curpage='
base_url = 'https://www.wayfair.com/furniture/sb0/sofas-c413892.html?itemsperpage=24&curpage='
#base_url = 'https://www.wayfair.com/filters/furniture/sb1/sofas-c413892-p2479~1000~*.html?itemsperpage=24&curpage='

i = 5779
#data from pages 1 - 200
for page in tqdm(range(1,201)):
    url = base_url + str(page)
    (product_links, image_links, prices) = get_product_info_from_mainpage(url)
    print(f"Number of products on Page {page}: {len(product_links)}\n") 
    for (product_url, img_url, price) in tqdm(zip(product_links, image_links, prices)):
        i += 1
        #item_no = f"{page}_{i}"
        item_no = f"{i}"
        try:
            info = {'price' : price, 'img_url' : img_url}
            product_info.append((item_no,info))     
        except:
            print(f"Failed. Product link: {product_url}")
            pass

  0%|                                                                                          | 0/200 [00:00<?, ?it/s]

Number of products scraped from the page:  24
Item: 1

Product link: https://www.wayfair.com/furniture/pdp/serta-futons-twin-661-tufted-back-convertible-sofa-stf1297.html

Image link: https://secure.img1-fg.wfcdn.com/im/57436246/resize-h310-w310%5Ecompr-r85/7194/71941621/Twin+66.1%2522+Tufted+Back+Convertible+Sofa.jpg

Price: from $172.99

Item: 2

Product link: https://www.wayfair.com/furniture/pdp/house-of-hampton-pasillas-6175-flared-arm-loveseat-w003177670.html?categoryid=413892&placement=1&slot=0&sponsoredid=cdd0fe761b7f4491538d3d9d552407d0691d87ddf2f85a092c71463e2b29d034&_txid=I%2BF9Ol8XEgit1zKrWz44Ag%3D%3D&isB2b=0&auctionId=318922e4-bccb-4419-b804-f0238225aee1

Image link: https://secure.img1-fg.wfcdn.com/im/13765528/resize-h310-w310%5Ecompr-r85/7101/71018521/Pasillas+61.75%2522+Flared+Arm+Loveseat.jpg

Price: from $509.99

Item: 3

Product link: https://www.wayfair.com/furniture/pdp/andover-mills-molina-full-74-futon-and-mattress-w003245840.html

Image link: https://secure.img1


20it [00:00, 20001.45it/s]
  0%|▍                                                                               | 1/200 [01:07<3:43:40, 67.44s/it]

Number of products scraped from the page:  24
Item: 1

Product link: https://www.wayfair.com/furniture/pdp/wade-logan-haysi-full-73-tufted-back-convertible-sofa-wdln2941.html

Image link: https://secure.img1-fg.wfcdn.com/im/40451803/resize-h310-w310%5Ecompr-r85/4017/40174738/Haysi+Full+73%2522+Tufted+Back+Convertible+Sofa.jpg

Price: $339.99

Item: 2

Product link: https://www.wayfair.com/furniture/pdp/hashtag-home-calvillo-standard-4975-flared-arm-loveseat-w001468120.html?categoryid=413892&placement=3&slot=0&sponsoredid=c45e3144342faee3c000c321e40e78c4b1c11d2301a834169cf05b227245aacb&_txid=I%2BF9Ol8XEkuya0oYb3HjAg%3D%3D&isB2b=0&auctionId=f4476941-1bc3-45e0-bdfb-8d9166ecefaf

Image link: https://secure.img1-fg.wfcdn.com/im/84885366/resize-h310-w310%5Ecompr-r85/8364/83642619/Calvillo+Standard+49.75%2522+Flared+Arm+Loveseat.jpg

Price: from $254.99

Item: 3

Product link: https://www.wayfair.com/furniture/pdp/serta-corwin-convertible-sleeper-xs10244.html

Image link: https://secure.img1-


20it [00:00, ?it/s]
  1%|▊                                                                               | 2/200 [02:20<3:48:24, 69.21s/it]

...


24it [00:00, ?it/s]
  4%|███▌                                                                            | 9/200 [11:05<4:10:10, 78.59s/it]

Number of products scraped from the page:  12
Item: 1

Product link: https://www.wayfair.com/furniture/pdp/ophelia-co-fincham-chesterfield-495-recessed-arm-loveseat-opco7940.html

Image link: https://secure.img1-fg.wfcdn.com/im/41689749/resize-h310-w310%5Ecompr-r85/1132/113222672/Fincham+Chesterfield+49.5%2522+Recessed+Arm+Loveseat.jpg

Price: $539.99

Item: 2

Product link: https://www.wayfair.com/furniture/pdp/andover-mills-talbert-61-rolled-arm-loveseat-w003241220.html

Image link: https://secure.img1-fg.wfcdn.com/im/18534962/resize-h310-w310%5Ecompr-r85/1184/118449259/Talbert+61%2522+Rolled+Arm+Loveseat.jpg

Price: $339.99

Item: 3

Product link: https://www.wayfair.com/furniture/pdp/house-of-hampton-letcher-velvet-63-square-arm-loveseat-w002991321.html

Image link: https://secure.img1-fg.wfcdn.com/im/92478873/resize-h310-w310%5Ecompr-r85/1109/110972783/Letcher+Velvet+63%2522+Square+Arm+Loveseat.jpg

Price: $489.99

Item: 4

Product link: https://www.wayfair.com/furniture/pdp/winst


8it [00:00, ?it/s]
  5%|███▉                                                                           | 10/200 [12:11<3:56:48, 74.78s/it]

Number of products scraped from the page:  12
Item: 1

Product link: https://www.wayfair.com/furniture/pdp/astoria-grand-moncalieri-93-flared-arms-sofa-astg1758.html

Image link: https://secure.img1-fg.wfcdn.com/im/39978620/resize-h310-w310%5Ecompr-r85/2705/27053749/Moncalieri+93%2522+Flared+Arms+Sofa.jpg

Price: $739.99

Item: 2

Product link: https://www.wayfair.com/furniture/pdp/andover-mills-talbert-61-rolled-arm-loveseat-w003241220.html

Image link: https://secure.img1-fg.wfcdn.com/im/18534962/resize-h310-w310%5Ecompr-r85/1184/118449259/Talbert+61%2522+Rolled+Arm+Loveseat.jpg

Price: $339.99

Item: 3

Product link: https://www.wayfair.com/furniture/pdp/house-of-hampton-letcher-velvet-63-square-arm-loveseat-w002991321.html

Image link: https://secure.img1-fg.wfcdn.com/im/92478873/resize-h310-w310%5Ecompr-r85/1109/110972783/Letcher+Velvet+63%2522+Square+Arm+Loveseat.jpg

Price: $489.99

Item: 4

Product link: https://www.wayfair.com/furniture/pdp/winston-porter-salomon-108-right-han


8it [00:00, ?it/s]
  6%|████▎                                                                          | 11/200 [13:18<3:49:00, 72.70s/it]

In [ ]:
save_data(product_info,1)
#product_info

In [ ]:
df = pd.read_csv('sofa_info_1.csv')
print(df.shape)
df.reset_index(drop = True, inplace = True)

In [ ]:
df

In [ ]:
df_new = save_images_from_csv(df)
df_new.to_csv('sofa.csv', index = False)

In [ ]:
url = 'https://secure.img1-fg.wfcdn.com/im/57436246/resize-h310-w310%5Ecompr-r85/7194/71941621/Twin+66.1%2522+Tufted+Back+Convertible+Sofa.jpg'
filepath = "C:/Users/Brian/Desktop/ThePriceIsRight/sofa/1.jpeg"
urllib.request.urlretrieve(url, filepath)

In [ ]:
df_new